In [1]:
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import pandas as pd
from torch import tensor
import numpy as np
from torch.utils.data import Dataset
from sklearn.metrics import mean_squared_error
import random
import os
import matplotlib.pyplot as plt
import pickle
from scipy.interpolate import interp1d
from torch.utils.data import DataLoader, random_split
import torch
#from torchsummary import summary
import seaborn as sns
import sys
import torch.nn.functional as F
import pywt
from sklearn.preprocessing import MinMaxScaler
from torch import FloatTensor
from scipy.stats import skew, kurtosis, entropy
from math import gcd
from functools import reduce
import pywt
from scipy.stats import skew, kurtosis
from scipy.special import entr

# Req for package
sys.path.append("../")
from SkinLearning.NN.Helpers import train, test, set_seed, DEVICE
from SkinLearning.NN.Models import MultiTemporal

In [2]:
a = {'a': 'a'}

In [155]:
if a:
    print("a")

a


In [2]:
set_seed()

In [3]:
# Folder name will correspond to index of sample
class SkinDataset(Dataset):
    def __init__(
        self,
        scaler,
        signalFolder="D:/SamplingResults2",
        sampleFile="../Data/newSamples.pkl",
        extraction_args={},
        runs=range(65535),
        steps=128,    
    ):
        # Load both disp1 and disp2 from each folder
        # Folders ordered according to index of sample
        # Use the corresponding sample as y -> append probe?
        self.input = []
        self.output = []
        
        with open(f"{sampleFile}", "rb") as f:
             samples = pickle.load(f)
        
        self.min = np.min(samples[runs])
        self.max = np.max(samples[runs])
        
        
        for run in tqdm(runs):
            inp = []
            fail = False
            
            files = os.listdir(f"{signalFolder}/{run}/")
            
            if files != ['Disp1.csv', 'Disp2.csv']:
                continue
            
            for file in files:
                a = pd.read_csv(f"{signalFolder}/{run}/{file}")
                a.rename(columns = {'0':'x', '0.1': 'y'}, inplace = True)
                
                # Skip if unconverged
                if a['x'].max() != 7.0:
                    fail = True
                    break

                # Interpolate curve for consistent x values
                xNew = np.linspace(0, 7, num=steps, endpoint=False)
                interped = interp1d(a['x'], a['y'], kind='cubic', fill_value="extrapolate")(xNew)
                    
                
                inp.append(interped.astype("float32"))
            
            if not fail:
                if len(inp) != 2:
                    raise Exception("sdf")

                self.input.append(inp)
                self.output.append(samples[int(run)])
        
        scaler.fit(self.output)
        self.output = scaler.fit_transform(self.output)
        self.output = tensor(self.output).type(FloatTensor)
        
        # Perform WPD on all sets of signals
        # Using the given parameters
        extracted_features = []
        for signals in self.input:
            extraction_args['signals'] = signals
            extracted_features.append(waveletExtraction(**extraction_args))
        self.input = extracted_features
        del extracted_features
            
        self.input = tensor(np.array(self.input)).type(FloatTensor)
        
        
    def __len__(self):
        return len(self.output)
    
    def __getitem__(self, idx):
        sample = {"input": self.input[idx], "output": self.output[idx]}
        return sample
    
    

In [4]:
"""
    Creates the data set from filtered samples
    Returns the dataset and the scaler
"""
def getDataset(**kwargs):
    # Get filtered data
    if not 'runs' in kwargs.keys():
        with open("../Data/filtered.pkl", "rb") as f:
            runs = pickle.load(f)

        kwargs['runs'] = runs

    scaler = MinMaxScaler()
    dataset = SkinDataset(scaler=scaler, **kwargs)

    return dataset, scaler

In [5]:
"""
    Creates a train/test split from the given data
    Returns train and test data loaders
"""
def getSplit(dataset, p1=0.8):
    train_n = int(p1 * len(dataset))
    test_n = len(dataset) - train_n
    train_set, test_set = random_split(dataset, [train_n, test_n])

    return DataLoader(train_set, batch_size=32, shuffle=True), \
        DataLoader(test_set, batch_size=32, shuffle=True)

In [6]:
def waveletExtraction(
    signals,
    method,
    combined=False,
    wavelet='db4',
    level=6,
    combine_method='concatenate',
    order='freq',
    levels=[6],
    stats=['mean', 'std', 'skew', 'kurtosis'],
    normalization=None,
):
    def get_statistics(coefficients, stats_list):
        features = []
        for stat in stats_list:
            if stat == 'mean':
                features.append(np.mean(coefficients))
            elif stat == 'std':
                features.append(np.std(coefficients))
            elif stat == 'skew':
                features.append(skew(coefficients))
            elif stat == 'kurtosis':
                features.append(kurtosis(coefficients))
        return features

    def extract_features_single_signal(signal, method, wavelet, level, order, levels, stats_list):
        wp = pywt.WaveletPacket(data=signal, wavelet=wavelet, mode='symmetric', maxlevel=level)
        features = []

        for l in levels:
            coeffs =  wp.get_level(level, order)
            coeffs = np.array([c.data for c in coeffs])
            
            if method == "e+s":
                for c in coeffs:
                    stats = get_statistics(c, stats_list)
                    stats.append(np.sum(np.square(c)))
                    features.extend(stats)
                

            if method == "energy" or method == "min-max":
                 # Normalise
                coeffs = (coeffs - np.mean(coeffs)) / np.std(coeffs)
                
            if method == 'raw':
                features.extend(np.concatenate(coeffs))
            elif method == 'energy':       
                features.extend([np.sum(np.square(c)) for c in coeffs])
            elif method == 'entropy':
                features.extend([np.sum(entr(np.abs(c))) for c in coeffs])
            elif method == 'min-max':
                features.extend([np.min(c) for c in coeffs] + [np.max(c) for c in coeffs])
            elif method == 'stats':
                for c in coeffs:
                    features.extend(get_statistics(c, stats_list))
        
            # Optional normalisation for raw and energy
            if normalization == "indvidual":
                features = (features - min(features)) / (max(features) - min(features))
                
        return features

    if combined:
        combined_signal = np.concatenate(signals, axis=0)
        combined_features = extract_features_single_signal(combined_signal, method, wavelet, level, order, levels, stats)
        features = combined_features
    else:
        features_list = [extract_features_single_signal(signal, method, wavelet, level, order, levels, stats) for signal in signals]

        if combine_method == 'concatenate':
            features = np.concatenate(features_list)
        elif combine_method == 'interleave':
            features = np.ravel(np.column_stack(features_list))
        else:
            raise ValueError("Invalid combine_method. Choose from 'concatenate' or 'interleave'.")
    
    if normalization == "combined":
        features = (features - np.min(features, axis=0)) / (np.max(features, axis=0) - np.min(features, axis=0))
    
    return features


In [7]:
def create_lengths_and_loaders(datasets, split_ratio=0.8, batch_size=32):
    lengths = {}
    loaders = {}

    for comb in datasets.keys():
        loaders[comb] = {}
        lengths[comb] = {}
        # Get train and test split
        
        for method in datasets[comb].keys():
            train_data, test_data = getSplit(datasets[comb][method][0], split_ratio)

            # Create DataLoaders
            train_loader, test_loader = getSplit(datasets[comb][method][0])

            # Update the loaders dictionary
            loaders[comb][method] = {'train': train_loader, 'test': test_loader}

            # Update the lengths dictionary
            lengths[comb][method] = len(datasets[comb][method][0][0]['input'])

    return lengths, loaders

In [40]:
def create_best(max_level=6):
    def parse_dataset(method, combined, order, normalize, stats):
        key = (method, normalize, stats)
        normalize = key[1]
        stats = key[-1]
        extraction_args = {
            "signals": None,
            "method": method,
            "combined": combined,
            "wavelet": "db4",
            "level": max_level,
            "order": order,
            "levels": [6],
            "normalization": normalize,
            "stats": stats if method == "stats" or method == "e+s" else None,
        }
        combined_str = 'combined' if combined else 'separated'
        dataset_dict[(combined_str, order)][key] = getDataset(
            extraction_args=extraction_args,
        )

    dataset_dict = {
        ("separated", "natural"): {},
    }

    
    #LSTM based on independent, ('separated', 'natural') ('raw', False, 'None')
    #- 93.3 MAPE, 0.048 MAE
    
    # LSTM based on concatenate, with dataset using ('separated', 'freq') ('entropy', 'None')
    #  93.8 MAPE, 0.049 MAE
    #STM based on independent, ('separated', 'natural') ('entropy', False, 'None')
    #- 93.6 MAPE, 0.049
    parse_dataset(
        "entropy",
        False,
        "natural",
        False,
        ('None')#('mean', 'std', 'skew', 'kurtosis')
    )
    
    return dataset_dict

all_datasets = create_best()
lengths, loaders = create_lengths_and_loaders(all_datasets)

100%|█████████████████████████████████████████████████████████████████████████████| 2241/2241 [00:10<00:00, 223.75it/s]


In [9]:
def test_models(model_type=None):
    temp_dict = {}
    for model_type in models.keys() if not model_type else [model_type]:
        for comb in models[model_type].keys():

            for ext_method in models[model_type][comb].keys():
                for fc_type in models[model_type][comb][ext_method].keys():

                    for model_method in models[model_type][comb][ext_method][fc_type].keys():
                        for output in models[model_type][comb][ext_method][fc_type][model_method].keys():
                            #print(f"Running {model_type} based on {model_method}, with dataset using {comb} {ext_method} using {output}")

                            validation = test(
                                    loaders[comb][ext_method]['test'],
                                    models[model_type][comb][ext_method][fc_type][model_method][output],
                                    all_datasets[comb][ext_method][1]
                            )
                            
                            """                            losses = comb_losses if comb == "combined" else non_comb_losses

                            print(
                                "Final training loss: ",
                                losses[model_type][comb][ext_method][fc_type][model_method]['Train Loss'][-1]
                            )"""
                            """print(
                                f"{fc_type}, MAPE: {validation[0]}, MAE: {validation[2]}\n"
                            )"""
                            
                            
                            try:
                                temp_dict[(model_type, comb, model_method, ext_method, output)][fc_type] = f"{fc_type}, MAPE: {validation[0]}, MAE: {validation[2]}"
                            except:
                                temp_dict[(model_type, comb, model_method, ext_method, output)] = {}
                                temp_dict[(model_type, comb, model_method, ext_method, output)][fc_type] = f"{fc_type}, MAPE: {validation[0]}, MAE: {validation[2]}"
    for key in temp_dict.keys():
        print(f"{key[0]} based on {key[2]}, with dataset using {key[1]} {key[3]} using {key[4]}")
        for strs in temp_dict[key].values():
              print(strs)
        print("\n")

In [10]:
def train_models(model_type=None, epochs=700, LR=0.001):
    comb_losses = {}
    non_comb_losses = {}
    
    for model_type in models.keys() if not model_type else [model_type]:
        non_comb_losses[model_type] = {}
        comb_losses[model_type] = {}

        
        for comb in models[model_type].keys():
            non_comb_losses[model_type][comb] = {}
            comb_losses[model_type][comb] = {}

            for ext_method in models[model_type][comb].keys():
                non_comb_losses[model_type][comb][ext_method] = {}
                comb_losses[model_type][comb][ext_method] = {}

                for fc_type in models[model_type][comb][ext_method].keys():
                    non_comb_losses[model_type][comb][ext_method][fc_type] = {}
                    comb_losses[model_type][comb][ext_method][fc_type] = {}
                    
                    for model_method in models[model_type][comb][ext_method][fc_type].keys():
                        non_comb_losses[model_type][comb][ext_method][fc_type][model_method] = {}
                        comb_losses[model_type][comb][ext_method][fc_type][model_method] = {}
                        
                        for output in models[model_type][comb][ext_method][fc_type][model_method].keys():

                            non_comb_losses[model_type][comb][ext_method][fc_type][model_method][output] = {}
                            comb_losses[model_type][comb][ext_method][fc_type][model_method][output] = {}
                            
                            print(f"Running {model_type} based on {model_method} and {fc_type}, with dataset using {comb} {ext_method}, with {output}")
                            train_loss, val_loss = train(
                                loaders[comb][ext_method]['train'],
                                models[model_type][comb][ext_method][fc_type][model_method][output],
                                val_loader=loaders[comb][ext_method]['test'],
                                LR=LR,
                                epochs=epochs,
                                early_stopping=True,
                                patience=50
                            )

                            # Stop at 50 epochs
                            # Only add if last lost had potential
                            if comb == "Combined":
                                comb_losses[model_type][comb][ext_method][fc_type][model_method][output] = {
                                    'Train Loss': train_loss,
                                    'Validation Loss': val_loss
                                }
                            else:
                                non_comb_losses[model_type][comb][ext_method][fc_type][model_method][output] = {
                                    'Train Loss': train_loss,
                                    'Validation Loss': val_loss
                                }
    return comb_losses, non_comb_losses

## Test

In [137]:
class Attention(nn.Module):
    def __init__(self, in_size, out):
        super(Attention, self).__init__()
        self.fc = nn.Linear(in_size, 1)  # Change output size to 1 for attention scores
        self.out = out
        print("In size:", in_size)

    def forward(self, lstm_outputs):
        if lstm_outputs.dim() == 2:
            lstm_outputs = lstm_outputs.unsqueeze(1)

        attention_scores = self.fc(lstm_outputs).squeeze(-1)  # Squeeze the last dimension to obtain attention scores
        attention_weights = torch.softmax(attention_scores, dim=-1)
        context_vector = torch.sum(attention_weights.unsqueeze(-1) * lstm_outputs, dim=-2)

        return context_vector, attention_weights

class MultiTemporal(nn.Module):
    def __init__(
        self,
        conv=True,
        input_size=15,
        hidden_size=256,
        single_fc=True,
        out="f_hidden",
        layers=1,
        temporal_type="RNN",
        fusion_method="concatenate",
    ):
        super(MultiTemporal, self).__init__()

        self.hidden_size = hidden_size
        self.out = out
        self.temporal_type = temporal_type
        self.fusion_method = fusion_method
        self.input_size = input_size
        self.conv = conv
        
        if conv:
            self.cnn = deepcopy(best_CNN)

        # Attention layer
        if fusion_method == "independent":
            input_size //= 2
        print("Input size:", input_size)

        if out == "f_output" or out == "f_hidden":
            attention_input_size = hidden_size
        elif out == "output" or out == "hidden":
            attention_input_size = hidden_size * 2
        elif out == "h+o":
            attention_input_size = hidden_size
     
            
        self.attention = Attention(attention_input_size, out)

        # Temporal layer
        if temporal_type == "RNN":
            net = nn.RNN
        elif temporal_type == "LSTM":
            net = nn.LSTM
        elif temporal_type == "GRU":
            net = nn.GRU
        else:
            raise ValueError("Invalid temporal_type. Choose from 'RNN', 'LSTM', or 'GRU'.")

        if fusion_method == 'concatenate':
            self.net = net(input_size, hidden_size, layers, batch_first=True)
        elif fusion_method == 'multi_channel':
            self.net = net(2, hidden_size, layers, batch_first=True)
        elif fusion_method == 'independent':
            self.net = net(input_size, hidden_size, layers, batch_first=True)
        else:
            raise ValueError("Invalid method. Choose from 'concatenate', 'multi_channel', or 'independent'.")

        # Fully connected layer(s)
        fc_in = hidden_size
        if out == "h+o" or fusion_method == "independent":
            fc_in *= 2

        if single_fc:
            self.fc = nn.Sequential(
                nn.Dropout(0.75),
                nn.Linear(fc_in, 6)
            )
        else:
            self.fc = nn.Sequential(
                nn.Linear(fc_in if fc_in < 256 else 256, 128),
                nn.ReLU(),
                nn.Linear(128, 64),
                nn.ReLU(),
                nn.Linear(64, 6),
            )
            
            if fc_in > 256:
                if fc_in > 1024:
                     init_layers = nn.Sequential(
                    nn.Linear(fc_in, 1024),
                    nn.ReLU(),
                    nn.Linear(1024, 512),
                    nn.ReLU(),
                    nn.Linear(512, 256),
                    nn.ReLU()
                    )
                elif fc_in > 512:
                    init_layers = nn.Sequential(
                    nn.Linear(fc_in, 512),
                    nn.ReLU(),
                    nn.Linear(512, 256),
                    nn.ReLU()
                    )
                else:    
                    init_layers = nn.Sequential(
                    nn.Linear(fc_in, 256),
                    nn.ReLU()
                    )

                self.fc = nn.Sequential(init_layers, self.fc)
        self.drop = nn.Dropout(0.25)
    def forward(self, x):
        batch_size = x.shape[0]
        
        if self.conv:
            x = self.cnn(x)
        else:
            x = x.reshape(batch_size, -1, self.input_size)

        def get_outputs(inp):
            if self.temporal_type == "LSTM":
                o, (h, _) = self.net(inp)
            else:
                o, h = self.net(inp)
            return o, h

        if self.fusion_method == 'independent':
            signal_size = self.input_size // 2
            x1 = x[..., :signal_size].reshape(batch_size, -1, signal_size)
            x2 = x[..., signal_size:].reshape(batch_size, -1, signal_size)
            o1, h1 = get_outputs(x1)
            o2, h2 = get_outputs(x2)
        else:
            o, h = get_outputs(x)

        if self.out == "f_hidden":
            if self.fusion_method == "independent":
                x = torch.cat([h1[-1], h2[-1]], dim=1).reshape(batch_size, -1)
            else:
                x = h[-1].reshape(batch_size, -1)
        elif self.out == "hidden":
            if self.fusion_method == "independent":
                x = torch.cat([h1, h2], dim=1).reshape(batch_size, -1)
            else:
                x = h.reshape(batch_size, -1)
        elif self.out == "f_output":
            if self.fusion_method == "independent":
                x = torch.cat([o1[:, -1, :], o2[:, -1, :]], dim=1).reshape(batch_size, -1)
            else:
                x = o[:, -1, :].reshape(batch_size, -1)
        elif self.out == "output":
            if self.fusion_method == "independent":
                x = torch.cat([o1, o2], dim=1).reshape(batch_size, -1)
            else:
                x = o.reshape(batch_size, -1)
        elif self.out == "h+o":
            if self.fusion_method == "independent":
                x1 = torch.cat([h1[-1], o1[:, -1, :]], dim=1)
                x2 = torch.cat([h2[-1], o2[:, -1, :]], dim=1)
                x = torch.cat([x1, x2], dim=1).view(batch_size, -1)
            else:
                x = torch.cat([h[-1], o[:, -1, :]], dim=1).view(batch_size, -1)

        # Apply attention mechanism
         # Apply attention mechanis
       # x = self.drop(x)
        #x, _ = self.attention(x)
        
        x = self.fc(x)
        return x

In [107]:
def createModel(temporal_type, combination, ext_method, fc, fusion_method, out):
    models = {}
    models[temporal_type] = {}
    models[temporal_type][combination] = {}
    models[temporal_type][combination][ext_method] = {}
    models[temporal_type][combination][ext_method][fc] = {}
    models[temporal_type][combination][ext_method][fc][fusion_method] = {}
    models[temporal_type][combination][ext_method][fc][fusion_method][out] = \
    MultiTemporal(
                    conv=False,
                    fusion_method=fusion_method,
                    input_size=lengths[combination][ext_method],
                    hidden_size=lengths[combination][ext_method]*2,
                    temporal_type=temporal_type,
                    single_fc=False,
                    out=out,
                    layers=1
                )
    return models

In [19]:
import gc
torch.cuda.empty_cache()
gc.collect()

19386

In [67]:
models = createModel('LSTM', ('separated','natural'), ('raw', False, ('None')), 'Multi FC', 'concatenate', 'output')

KeyError: ('raw', False, 'None')

In [29]:
train_models()

Running LSTM based on concatenate and Multi FC, with dataset using ('separated', 'natural') ('raw', False, 'None'), with output
Using: cuda


100%|████████████████████████| 56/56 [00:00<00:00, 95.12batch/s, counter=3, epoch=135, lastLoss=0.0664, valLoss=0.0753]


100%|████████████████████████| 56/56 [00:00<00:00, 90.47batch/s, counter=34, epoch=271, lastLoss=0.045, valLoss=0.0569]


100%|███████████████████████| 56/56 [00:00<00:00, 97.26batch/s, counter=49, epoch=286, lastLoss=0.0444, valLoss=0.0691]


Early stopping after 287 epochs
Average train loss: 0.028140812053639747
Average validation loss: 0.031218803576947677


({'LSTM': {('separated',
    'natural'): {('raw',
     False,
     'None'): {'Multi FC': {'concatenate': {'output': {}}}}}}},
 {'LSTM': {('separated',
    'natural'): {('raw',
     False,
     'None'): {'Multi FC': {'concatenate': {'output': {'Train Loss': [0.22550297182585513,
         0.17957052721508912,
         0.17205837395574367,
         0.1579191189791475,
         0.14884042367339134,
         0.14199346889342582,
         0.1380731901153922,
         0.1343712793397052,
         0.13160848218415463,
         0.12956672294863633,
         0.12736466154456139,
         0.12379842610763651,
         0.11367936259401697,
         0.10851163736411504,
         0.10320113506168127,
         0.10165151980306421,
         0.09752368993524994,
         0.09612128231674433,
         0.09590492118149996,
         0.09570212475955486,
         0.09382437555385488,
         0.0946454130379217,
         0.09281867223658732,
         0.09089988323726825,
         0.09325016888656787,
     

In [108]:
models = createModel('LSTM', ('separated','natural'), ('stats', False, ('mean', 'std', 'skew', 'kurtosis')), 'Multi FC', 'concatenate', 'output')

KeyError: ('stats', False, ('mean', 'std', 'skew', 'kurtosis'))

In [69]:
train_models()

Running LSTM based on independent and Multi FC, with dataset using ('separated', 'natural') ('stats', False, ('mean', 'std', 'skew', 'kurtosis')), with output
Using: cuda


100%|██████████████████████| 56/56 [00:00<00:00, 171.11batch/s, counter=28, epoch=135, lastLoss=0.0335, valLoss=0.0756]


100%|██████████████████████| 56/56 [00:00<00:00, 164.90batch/s, counter=49, epoch=156, lastLoss=0.0307, valLoss=0.0746]

Early stopping after 157 epochs
Average train loss: 0.012182126712643217
Average validation loss: 0.01763009819947183


({'LSTM': {('separated',
    'natural'): {('stats',
     False,
     ('mean',
      'std',
      'skew',
      'kurtosis')): {'Multi FC': {'independent': {'output': {}}}}}}},
 {'LSTM': {('separated',
    'natural'): {('stats',
     False,
     ('mean',
      'std',
      'skew',
      'kurtosis')): {'Multi FC': {'independent': {'output': {'Train Loss': [0.17554884417248623,
         0.12397197554154056,
         0.11586982025099653,
         0.10673946913863931,
         0.10079988384885448,
         0.0991678571860705,
         0.0960657796157258,
         0.09344793111085892,
         0.09321171843579837,
         0.0920246352574655,
         0.09072907641530037,
         0.08817422762513161,
         0.08874306521777596,
         0.08442527893930674,
         0.0835856937670282,
         0.0853770582803658,
         0.08234439085104636,
         0.08101382545594658,
         0.07959951081180147,
         0.08124725693570715,
         0.08127463436020273,
         0.07675961671130997

In [116]:
models = createModel('LSTM', ('separated','natural'), ('energy', False, ('None')), 'Multi FC', 'independent', 'f_hidden')

KeyError: ('energy', False, 'None')

In [84]:
train_models()

Running LSTM based on independent and Multi FC, with dataset using ('separated', 'natural') ('energy', False, 'None'), with f_hidden
Using: cuda


100%|██████████████████████| 56/56 [00:00<00:00, 193.36batch/s, counter=34, epoch=135, lastLoss=0.0856, valLoss=0.0967]


100%|███████████████████████| 56/56 [00:00<00:00, 180.94batch/s, counter=6, epoch=271, lastLoss=0.0734, valLoss=0.0846]


100%|██████████████████████| 56/56 [00:00<00:00, 193.27batch/s, counter=30, epoch=407, lastLoss=0.0632, valLoss=0.0752]


100%|██████████████████████| 56/56 [00:00<00:00, 190.50batch/s, counter=49, epoch=426, lastLoss=0.0606, valLoss=0.0781]

Early stopping after 427 epochs
Average train loss: 0.04824699130156363
Average validation loss: 0.055285115343651596


({'LSTM': {('separated',
    'natural'): {('energy',
     False,
     'None'): {'Multi FC': {'independent': {'f_hidden': {}}}}}}},
 {'LSTM': {('separated',
    'natural'): {('energy',
     False,
     'None'): {'Multi FC': {'independent': {'f_hidden': {'Train Loss': [0.19008259076092923,
         0.16656623275152274,
         0.15580747782119683,
         0.1426835838439209,
         0.13364120785679137,
         0.12935950926371984,
         0.12355750692742211,
         0.1212962796645505,
         0.11676650467727866,
         0.11381954646536283,
         0.11236824880221061,
         0.11092009861022234,
         0.10599850464080061,
         0.10470631425934178,
         0.1041876987687179,
         0.1041454052818673,
         0.10450398203517709,
         0.10303006919899158,
         0.10295912317399468,
         0.10010482211198125,
         0.10119035627160754,
         0.10146769721593175,
         0.09985730690615517,
         0.09954828502876419,
         0.10038338774549

# Doesn't help raw or combined entropy + statistics, statistics or energy

In [109]:
models = createModel('LSTM', ('separated','natural'), ('entropy', False, ('None')), 'Multi FC', 'concatenate', 'output')

Input size: 128
In size: 256


In [110]:
train_models()

Running LSTM based on concatenate and Multi FC, with dataset using ('separated', 'natural') ('entropy', False, 'None'), with output
Using: cuda


100%|███████████████████████| 56/56 [00:00<00:00, 226.79batch/s, counter=3, epoch=135, lastLoss=0.0625, valLoss=0.0667]


100%|██████████████████████| 56/56 [00:00<00:00, 234.31batch/s, counter=31, epoch=271, lastLoss=0.0484, valLoss=0.0558]


100%|███████████████████████| 56/56 [00:00<00:00, 257.77batch/s, counter=1, epoch=407, lastLoss=0.0463, valLoss=0.0529]


100%|██████████████████████| 56/56 [00:00<00:00, 248.67batch/s, counter=49, epoch=503, lastLoss=0.0413, valLoss=0.0531]

Early stopping after 504 epochs
Average train loss: 0.04235270744208627
Average validation loss: 0.04640381559055476


({'LSTM': {('separated',
    'natural'): {('entropy',
     False,
     'None'): {'Multi FC': {'concatenate': {'output': {}}}}}}},
 {'LSTM': {('separated',
    'natural'): {('entropy',
     False,
     'None'): {'Multi FC': {'concatenate': {'output': {'Train Loss': [0.23901719174214772,
         0.17941193176167353,
         0.16434138720589025,
         0.156950337014028,
         0.15339345990547112,
         0.1478658084358488,
         0.13935656512954406,
         0.12343222341899361,
         0.11660712491720915,
         0.11305423599800893,
         0.10962047081972871,
         0.10647250725222486,
         0.10422900239271778,
         0.10174264346382447,
         0.10116600072277444,
         0.09868544007518462,
         0.09692891780287027,
         0.09614491449402911,
         0.09555258389030184,
         0.09507195744663477,
         0.09621053215648447,
         0.09437353922320264,
         0.09360869787633419,
         0.09291344455310277,
         0.092244663142732

In [111]:
test_models()

LSTM based on concatenate, with dataset using ('separated', 'natural') ('entropy', False, 'None') using output
Multi FC, MAPE: 6.915175437927246, MAE: 0.05064303477605184




In [128]:
models = createModel('LSTM', ('separated','natural'), ('entropy', False, 'None'), 'Multi FC', 'independent', 'f_hidden')

Input size: 64
In size: 512


In [129]:
train_models()

Running LSTM based on independent and Multi FC, with dataset using ('separated', 'natural') ('entropy', False, 'None'), with f_hidden
Using: cuda


100%|████████████████████████| 56/56 [00:00<00:00, 170.99batch/s, counter=1, epoch=135, lastLoss=0.0521, valLoss=0.054]


100%|███████████████████████| 56/56 [00:00<00:00, 186.05batch/s, counter=8, epoch=271, lastLoss=0.0416, valLoss=0.0483]


100%|██████████████████████| 56/56 [00:00<00:00, 163.02batch/s, counter=29, epoch=407, lastLoss=0.0354, valLoss=0.0407]


100%|██████████████████████| 56/56 [00:00<00:00, 179.49batch/s, counter=23, epoch=543, lastLoss=0.0295, valLoss=0.0408]


100%|██████████████████████| 56/56 [00:00<00:00, 168.82batch/s, counter=49, epoch=569, lastLoss=0.0312, valLoss=0.0424]

Early stopping after 570 epochs
Average train loss: 0.03879492533839859
Average validation loss: 0.04520162288613973


({'LSTM': {('separated',
    'natural'): {('entropy',
     False,
     'None'): {'Multi FC': {'independent': {'f_hidden': {}}}}}}},
 {'LSTM': {('separated',
    'natural'): {('entropy',
     False,
     'None'): {'Multi FC': {'independent': {'f_hidden': {'Train Loss': [0.2301958099539791,
         0.1830701506031411,
         0.16941582784056664,
         0.16068570635148458,
         0.1539521808070796,
         0.14452438881354673,
         0.1305399328204138,
         0.12136588831033025,
         0.11863596245114293,
         0.11263880013887372,
         0.11005106994083949,
         0.10748614197863,
         0.10489377632204976,
         0.10277512217206615,
         0.10005681695682662,
         0.09918922505208425,
         0.09759495527084384,
         0.09725251354809318,
         0.09644971481923546,
         0.0955904864573053,
         0.09530956032020706,
         0.09391417700265135,
         0.09268418499933821,
         0.09224530096564974,
         0.0922343257282461

In [130]:
test_models()

LSTM based on independent, with dataset using ('separated', 'natural') ('entropy', False, 'None') using f_hidden
Multi FC, MAPE: 5.5103759765625, MAE: 0.041255035003026325




In [134]:
models = createModel('GRU', ('separated','natural'), ('entropy', False, 'None'), 'Multi FC', 'concatenate', 'f_hidden')

Input size: 128
In size: 256


In [135]:
train_models()

Running GRU based on concatenate and Multi FC, with dataset using ('separated', 'natural') ('entropy', False, 'None'), with f_hidden
Using: cuda


100%|███████████████████████| 56/56 [00:00<00:00, 233.81batch/s, counter=4, epoch=135, lastLoss=0.0591, valLoss=0.0655]


100%|███████████████████████| 56/56 [00:00<00:00, 147.18batch/s, counter=7, epoch=271, lastLoss=0.0498, valLoss=0.0564]


100%|██████████████████████| 56/56 [00:00<00:00, 246.74batch/s, counter=49, epoch=367, lastLoss=0.0453, valLoss=0.0542]

Early stopping after 368 epochs
Average train loss: 0.03290477253064247
Average validation loss: 0.035256785733093106


({'GRU': {('separated',
    'natural'): {('entropy',
     False,
     'None'): {'Multi FC': {'concatenate': {'f_hidden': {}}}}}}},
 {'GRU': {('separated',
    'natural'): {('entropy',
     False,
     'None'): {'Multi FC': {'concatenate': {'f_hidden': {'Train Loss': [0.21768961261425698,
         0.17332812851028784,
         0.1581172360373395,
         0.15077520135257924,
         0.14481900792036737,
         0.13663444016128778,
         0.12313774108354535,
         0.11459734663367271,
         0.1105504475001778,
         0.1070837773648756,
         0.10322788065033299,
         0.10039053577929735,
         0.0999929578974843,
         0.09769443741866521,
         0.09637953686927046,
         0.09661912279469627,
         0.09504602317299161,
         0.0938966114606176,
         0.09399680354233299,
         0.09327302487300974,
         0.09148418517517191,
         0.09130216270153012,
         0.09065157654029983,
         0.09060923156461545,
         0.090312697259443

In [136]:
test_models()

GRU based on concatenate, with dataset using ('separated', 'natural') ('entropy', False, 'None') using f_hidden
Multi FC, MAPE: 7.156386852264404, MAE: 0.054024173567692436




# Prev

In [138]:
models = createModel('LSTM', ('separated','natural'), ('entropy', False, ('None')), 'Multi FC', 'concatenate', 'output')

Input size: 128
In size: 512


In [139]:
train_models()

Running LSTM based on concatenate and Multi FC, with dataset using ('separated', 'natural') ('entropy', False, 'None'), with output
Using: cuda


100%|███████████████████████| 56/56 [00:00<00:00, 308.47batch/s, counter=0, epoch=135, lastLoss=0.0612, valLoss=0.0644]


100%|███████████████████████| 56/56 [00:00<00:00, 282.84batch/s, counter=5, epoch=271, lastLoss=0.0494, valLoss=0.0559]


100%|███████████████████████| 56/56 [00:00<00:00, 281.45batch/s, counter=8, epoch=407, lastLoss=0.0443, valLoss=0.0527]


100%|██████████████████████| 56/56 [00:00<00:00, 297.34batch/s, counter=49, epoch=448, lastLoss=0.0441, valLoss=0.0534]

Early stopping after 449 epochs
Average train loss: 0.039487192629247304
Average validation loss: 0.04264413314649747


({'LSTM': {('separated',
    'natural'): {('entropy',
     False,
     'None'): {'Multi FC': {'concatenate': {'output': {}}}}}}},
 {'LSTM': {('separated',
    'natural'): {('entropy',
     False,
     'None'): {'Multi FC': {'concatenate': {'output': {'Train Loss': [0.2261670274393899,
         0.1790232083627156,
         0.16131953788655146,
         0.15578601988298552,
         0.1522763238421508,
         0.1464954779616424,
         0.14100226879652059,
         0.13534213456192187,
         0.12113753293773957,
         0.11251207361263889,
         0.1079171085730195,
         0.10394364103142704,
         0.10251974487411124,
         0.10007311736366578,
         0.0996256966942123,
         0.09888393245637417,
         0.09739012683608703,
         0.09527164178767375,
         0.0952481719266091,
         0.09627693252904075,
         0.09327036967234952,
         0.09274476593626398,
         0.09252036682197026,
         0.0924461938972984,
         0.09216367253767592,
 

In [140]:
test_models()

LSTM based on concatenate, with dataset using ('separated', 'natural') ('entropy', False, 'None') using output
Multi FC, MAPE: 6.507602214813232, MAE: 0.04831048659980297




In [141]:
models = createModel('LSTM', ('separated','natural'), ('entropy', False, 'None'), 'Multi FC', 'independent', 'f_hidden')

Input size: 64
In size: 256


In [142]:
train_models()

Running LSTM based on independent and Multi FC, with dataset using ('separated', 'natural') ('entropy', False, 'None'), with f_hidden
Using: cuda


100%|██████████████████████| 56/56 [00:00<00:00, 180.07batch/s, counter=12, epoch=135, lastLoss=0.0473, valLoss=0.0624]


100%|███████████████████████| 56/56 [00:00<00:00, 200.72batch/s, counter=6, epoch=271, lastLoss=0.0356, valLoss=0.0481]


100%|███████████████████████| 56/56 [00:00<00:00, 214.92batch/s, counter=29, epoch=407, lastLoss=0.031, valLoss=0.0517]


100%|███████████████████████| 56/56 [00:00<00:00, 196.41batch/s, counter=49, epoch=501, lastLoss=0.0249, valLoss=0.044]

Early stopping after 502 epochs
Average train loss: 0.032225124352678124
Average validation loss: 0.038969223606777154


({'LSTM': {('separated',
    'natural'): {('entropy',
     False,
     'None'): {'Multi FC': {'independent': {'f_hidden': {}}}}}}},
 {'LSTM': {('separated',
    'natural'): {('entropy',
     False,
     'None'): {'Multi FC': {'independent': {'f_hidden': {'Train Loss': [0.22343485270227706,
         0.17904655369264738,
         0.16657590786261217,
         0.16018611006438732,
         0.155927767178842,
         0.1495366892112153,
         0.13287382306797163,
         0.11750525049865246,
         0.11244636055614267,
         0.10934272421790021,
         0.10476832703820296,
         0.10325323098472186,
         0.10088185739836522,
         0.09897256802235331,
         0.09713855585349458,
         0.09651125967502594,
         0.09404937656862396,
         0.09308039676398039,
         0.09292076315198626,
         0.09179803462965148,
         0.09020233965877976,
         0.09030391422233411,
         0.09037077546651874,
         0.09001918097159692,
         0.08901680713

In [143]:
test_models()

LSTM based on independent, with dataset using ('separated', 'natural') ('entropy', False, 'None') using f_hidden
Multi FC, MAPE: 6.031959056854248, MAE: 0.04471707865595818




In [144]:
models = createModel('GRU', ('separated','natural'), ('entropy', False, 'None'), 'Multi FC', 'concatenate', 'f_hidden')

Input size: 128
In size: 256


In [145]:
train_models()

Running GRU based on concatenate and Multi FC, with dataset using ('separated', 'natural') ('entropy', False, 'None'), with f_hidden
Using: cuda


100%|███████████████████████| 56/56 [00:00<00:00, 267.85batch/s, counter=3, epoch=135, lastLoss=0.0578, valLoss=0.0723]


100%|████████████████████████| 56/56 [00:00<00:00, 286.35batch/s, counter=4, epoch=271, lastLoss=0.048, valLoss=0.0542]


100%|██████████████████████| 56/56 [00:00<00:00, 255.14batch/s, counter=34, epoch=407, lastLoss=0.0444, valLoss=0.0508]


100%|██████████████████████| 56/56 [00:00<00:00, 288.60batch/s, counter=16, epoch=543, lastLoss=0.0426, valLoss=0.0475]


100%|███████████████████████| 56/56 [00:00<00:00, 205.51batch/s, counter=49, epoch=576, lastLoss=0.0408, valLoss=0.054]


Early stopping after 577 epochs
Average train loss: 0.046168544958820765
Average validation loss: 0.05099742307518387


({'GRU': {('separated',
    'natural'): {('entropy',
     False,
     'None'): {'Multi FC': {'concatenate': {'f_hidden': {}}}}}}},
 {'GRU': {('separated',
    'natural'): {('entropy',
     False,
     'None'): {'Multi FC': {'concatenate': {'f_hidden': {'Train Loss': [0.24736844162855828,
         0.17595549380140646,
         0.1593099758028984,
         0.1545197678995984,
         0.1489961684814521,
         0.13959652903888906,
         0.129346627609006,
         0.11974762286990881,
         0.1151147016457149,
         0.11143031397036143,
         0.10867659521422215,
         0.10625384947551149,
         0.10363843478262424,
         0.10187999292143754,
         0.10041908866592816,
         0.09842591426734414,
         0.09740329666861466,
         0.09776894841343164,
         0.09626834600099496,
         0.09512349124997854,
         0.09488119038620166,
         0.09347818605601788,
         0.09356461092829704,
         0.09292652271687984,
         0.0924681685864925

In [146]:
test_models()

GRU based on concatenate, with dataset using ('separated', 'natural') ('entropy', False, 'None') using f_hidden
Multi FC, MAPE: 6.3538384437561035, MAE: 0.04683882718284925


